In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Schizophrenia"
cohort = "GSE193818"

# Input paths
in_trait_dir = "../../input/GEO/Schizophrenia"
in_cohort_dir = "../../input/GEO/Schizophrenia/GSE193818"

# Output paths
out_data_file = "../../output/preprocess/Schizophrenia/GSE193818.csv"
out_gene_data_file = "../../output/preprocess/Schizophrenia/gene_data/GSE193818.csv"
out_clinical_data_file = "../../output/preprocess/Schizophrenia/clinical_data/GSE193818.csv"
json_path = "../../output/preprocess/Schizophrenia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from cohort of remitted first-episode schizophrenia patients with less than 5 years of evolution"
!Series_summary	"To explore the genomic architecture of schizophrenia symptomatology, we analysed blood co-expression modules, i.e. clusters of genes with highly correlated expression, in a cohort of remitted first-episode schizophrenia patients with less than 5 years of evolution, and their association with clinical data, including global functioning, clinical symptomatology and premorbid adjustment."
!Series_overall_design	"91 samples of RNA isolated from peripheral blood collected with PAXgene tubes were analyzed using  weighted gene co-expression network analysis"
Sample Characteristics Dictionary:
{0: ['gender: Female', 'gender: Male', 'gender: NA'], 1: ['age: 30', 'age: 23', 'age: 35', 'age: 20', 'age: 29', 'age: 21', 'age: 19', 'age: 22', 'age: 27', 'age: 24', 'age: 28', 'age: 37', 'age: 38', 'age: 32', 'age: 34', 'age: 17', 'ag

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on background information: "Expression data from cohort", "91 samples of RNA isolated"
# This indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Schizophrenia):
# From the background information, we know this is a study on schizophrenia patients
# The study mentions analyzing "association with clinical data, including global functioning, 
# clinical symptomatology and premorbid adjustment" - these could be trait variables
# However, none of these are explicitly shown in the sample characteristics dictionary
# Since we don't see a clear trait variable in the available keys, we'll mark as not available
trait_row = None

# For age:
# Key 1 contains age data: 'age: 30', 'age: 23', etc.
age_row = 1

# For gender:
# Key 0 contains gender data: 'gender: Female', 'gender: Male'
gender_row = 0

# 2.2 Data Type Conversion Functions

# For trait (defined for completeness)
def convert_trait(value):
    if pd.isna(value):
        return None
    try:
        trait_str = value.split(':', 1)[1].strip()
        if trait_str == 'NA':
            return None
        return float(trait_str)  # Assuming it would be a continuous measure
    except:
        return None

# For age
def convert_age(value):
    if pd.isna(value):
        return None
    try:
        age_str = value.split(':', 1)[1].strip()
        if age_str == 'NA':
            return None
        return float(age_str)  # Continuous value
    except:
        return None

# For gender
def convert_gender(value):
    if pd.isna(value):
        return None
    try:
        gender_str = value.split(':', 1)[1].strip()
        if gender_str == 'NA':
            return None
        # Convert to binary: 0 for Female, 1 for Male
        if gender_str.lower() == 'female':
            return 0
        elif gender_str.lower() == 'male':
            return 1
        else:
            return None
    except:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering validation
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep entirely as instructed


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Schizophrenia/GSE193818/GSE193818_series_matrix.txt.gz


Gene data shape: (24351, 91)
First 20 gene/probe identifiers:
Index(['AFFX-BkGr-GC03_st', 'AFFX-BkGr-GC04_st', 'AFFX-BkGr-GC05_st',
       'AFFX-BkGr-GC06_st', 'AFFX-BkGr-GC07_st', 'AFFX-BkGr-GC08_st',
       'AFFX-BkGr-GC09_st', 'AFFX-BkGr-GC10_st', 'AFFX-BkGr-GC11_st',
       'AFFX-BkGr-GC12_st', 'AFFX-BkGr-GC13_st', 'AFFX-BkGr-GC14_st',
       'AFFX-BkGr-GC15_st', 'AFFX-BkGr-GC16_st', 'AFFX-BkGr-GC17_st',
       'AFFX-BkGr-GC18_st', 'AFFX-BkGr-GC19_st', 'AFFX-BkGr-GC20_st',
       'AFFX-BkGr-GC21_st', 'AFFX-BkGr-GC22_st'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, I can see they start with 'AFFX-' which indicates they are
# Affymetrix probe IDs from a microarray platform, not standard human gene symbols.
# These need to be mapped to gene symbols for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the SPOT_ID.1 column which appears to contain gene names
print("\nAnalyzing SPOT_ID.1 column for gene symbols:")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Extract a few sample values
    sample_values = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, value in enumerate(sample_values):
        print(f"Sample {i+1} excerpt: {value[:200]}...")  # Print first 200 chars
        # Test the extract_human_gene_symbols function on these values
        symbols = extract_human_gene_symbols(value)
        print(f"  Extracted gene symbols: {symbols}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0].split('_')[0]  # Get prefix of first gene ID
print(f"\nGene data ID prefix: {gene_data_id_prefix}")

# Look for columns that might match the gene data IDs
for col in gene_annotation.columns:
    if gene_annotation[col].astype(str).str.contains(gene_data_id_prefix).any():
        print(f"Column '{col}' contains values matching gene data ID pattern")

# Check if there's any column that might contain transcript or gene IDs
print("\nChecking for columns containing transcript or gene related terms:")
for col in gene_annotation.columns:
    if any(term in col.upper() for term in ['GENE', 'TRANSCRIPT', 'SYMBOL', 'NAME', 'DESCRIPTION']):
        print(f"Column '{col}' may contain gene-related information")
        # Show sample values
        print(f"Sample values: {gene_annotation[col].head(3).tolist()}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'category', 'SPOT_ID', 'SPOT_ID.1']
{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'probeset_id': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['69091', '924880', '960587', '966497', '1001138'], 'stop': ['70008', '944581', '965719', '975865', '1014541'], 'total_probes': [10.0, 10.0, 10.0, 10.0, 10.0], 'category': ['main', 'main', 'main', 'main', 'main'], 'SPOT_ID': ['Coding', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex'], 'SPOT_ID.1': ['NM_001005484 // RefSeq // Homo sapiens olfactory receptor, family 4, subfamily F, member 5 (OR4F5), mRNA. // chr1 // 100 // 100 // 0 // --- // 0 /// ENST00000335137 //

Column 'ID' contains values matching gene data ID pattern



Checking for columns containing transcript or gene related terms:
Column 'seqname' may contain gene-related information
Sample values: ['chr1', 'chr1', 'chr1']


### Step 6: Gene Identifier Mapping

In [7]:
# The current gene expression data (gene_data) contains Affymetrix probe IDs like 'AFFX-BkGr-GC03_st'
# From analyzing the gene annotation dataframe, we see:
# 1. The probeset_id column in gene_annotation doesn't match the format in gene_data
# 2. The SPOT_ID.1 column contains gene mapping information with gene symbols

# But we need to check if there's a direct probe ID mapping column that matches gene_data's IDs
# Let's check for matching IDs between gene_data and a possible ID list file

# 1. Look for a separate file that might have probe-to-gene mapping
try:
    # Look for a file with platform information that might contain the mappings
    # Common files in GEO datasets include platform annotation files
    platform_files = [f for f in os.listdir(in_cohort_dir) if 'platform' in f.lower() or 'annot' in f.lower()]
    
    if platform_files:
        platform_file = os.path.join(in_cohort_dir, platform_files[0])
        print(f"Found potential platform annotation file: {platform_file}")
        # Read the platform file - this would typically contain probe ID to gene mapping
        platform_df = pd.read_csv(platform_file, sep='\t', comment='#')
        print(f"Platform file columns: {platform_df.columns.tolist()}")
        
        # If we found a platform file with proper mapping, use it directly
        if 'ID' in platform_df.columns and any('gene' in col.lower() for col in platform_df.columns):
            gene_col = next(col for col in platform_df.columns if 'gene' in col.lower() and 'symbol' in col.lower())
            mapping_df = platform_df[['ID', gene_col]].rename(columns={gene_col: 'Gene'})
            print(f"Using direct mapping from platform file with columns: ID and {gene_col}")
        else:
            # If no suitable platform file, we'll need to use the SOFT file annotations
            raise FileNotFoundError("Platform file doesn't contain required mapping")
    else:
        raise FileNotFoundError("No platform annotation file found")
        
except (FileNotFoundError, Exception) as e:
    print(f"Using SOFT file for gene mapping: {e}")
    # Since we need to map probe IDs to gene symbols, and the SPOT_ID.1 column contains gene information,
    # we need to extract gene symbols from this column
    
    # Create a mapping between probe IDs and gene symbols using the extract_human_gene_symbols function
    gene_annotation['Gene'] = gene_annotation['SPOT_ID.1'].apply(extract_human_gene_symbols)
    
    # Since the first gene in the list is typically the main gene for that probe,
    # We'll try to look for an exact match between gene_data index and a column in gene_annotation
    
    # Test if we can find direct mapping between gene_data index and gene_annotation
    sample_gene_ids = list(gene_data.index[:5])
    print(f"Sample gene IDs from expression data: {sample_gene_ids}")
    
    # Look for matching ID patterns in gene_annotation
    match_found = False
    mapping_df = None
    
    # Let's see if we can match the Affymetrix IDs in the SOFT file
    # First, try direct match with ID column
    for col in gene_annotation.columns:
        # Check first few values to see if format matches
        if gene_annotation[col].astype(str).str.contains('_st$').any():
            print(f"Potential match found in column {col}")
            # See if the ID patterns match between gene_data and this column
            match_found = True
            mapping_df = get_gene_mapping(gene_annotation, col, 'Gene')
            break
    
    # If no direct match found, we need to use an alternative approach
    if not match_found:
        print("No direct match found for gene IDs. Using alternative mapping approach.")
        # Since the annotation doesn't directly contain the probe IDs in the format we need,
        # we'll try to match with a platform-specific annotation file or use a more complex approach
        
        # For now, let's create a mapping based on the transcript/gene information in SPOT_ID.1
        gene_annotation['Gene'] = gene_annotation['SPOT_ID.1'].apply(extract_human_gene_symbols)
        # Keep only rows with at least one gene symbol
        gene_annotation = gene_annotation[gene_annotation['Gene'].apply(lambda x: len(x) > 0)]
        # Create mapping from ID column to Gene
        mapping_df = get_gene_mapping(gene_annotation, 'ID', 'Gene')

# Print the mapping dataframe
print("\nGene mapping dataframe preview:")
print(mapping_df.head())
print(f"Mapping shape: {mapping_df.shape}")

# Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"\nConverted gene data shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])

# Normalize gene symbols to handle synonyms and case differences
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"\nGene data after normalization: {gene_data.shape}")

# Save the processed gene data to a file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

Using SOFT file for gene mapping: No platform annotation file found


Sample gene IDs from expression data: ['AFFX-BkGr-GC03_st', 'AFFX-BkGr-GC04_st', 'AFFX-BkGr-GC05_st', 'AFFX-BkGr-GC06_st', 'AFFX-BkGr-GC07_st']


Potential match found in column ID



Gene mapping dataframe preview:
                  ID                                               Gene
0  TC0100006437.hg.1            [OR4F5, ENSEMBL, UCSC, CCDS30547, HGNC]
1  TC0100006476.hg.1  [SAMD11, ENSEMBL, BC024295, MGC, IMAGE, BC0332...
2  TC0100006479.hg.1  [KLHL17, ENSEMBL, BC166618, IMAGE, MGC, CCDS30...
3  TC0100006480.hg.1  [PLEKHN1, ENSEMBL, BC101386, MGC, IMAGE, BC101...
4  TC0100006483.hg.1  [ISG15, ENSEMBL, BC009507, MGC, IMAGE, CCDS6, ...
Mapping shape: (2243221, 2)



Converted gene data shape: (0, 91)
First few gene symbols:
Index([], dtype='object', name='Gene')

Gene data after normalization: (0, 91)
Gene data saved to ../../output/preprocess/Schizophrenia/gene_data/GSE193818.csv
